## Homework

### Dataset

In this homework, we'll deploy the dino or dragon model we trained in the [previous homework](../09-serverless/homework.md).

Download the model from here:

https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

In [1]:
!wget -nc https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

File ‘dino_dragon_10_0.899.h5’ already there; not retrieving.



In [2]:
#@ IMPORTING LIBRARIES AND DEPENDENCIES:
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

In [3]:
model = keras.models.load_model("dino_dragon_10_0.899.h5")

2022-11-29 00:33:09.684829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-29 00:33:09.693593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2022-11-29 00:33:09.693624: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-29 00:33:09.694365: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimi

In [4]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the saved model
tflite_model = converter.convert()

# Save the model.
with open('dino-dragon-model.tflite', 'wb') as f:
  f.write(tflite_model)

2022-11-29 00:33:10.419267: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpl65_cxp8/assets


2022-11-29 00:33:11.037025: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-11-29 00:33:11.037069: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-11-29 00:33:11.037646: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpl65_cxp8
2022-11-29 00:33:11.038432: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-11-29 00:33:11.038445: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpl65_cxp8
2022-11-29 00:33:11.041979: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-11-29 00:33:11.098384: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpl65_cxp8
2022-11-29 00:33:11.107200: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 69557 m

In [5]:
!ls -lh

total 129M
-rwxrwxrwx 1 andre andre  269 Nov 29 00:22 Dockerfile
-rwxrwxrwx 1 andre andre 368K Mar  4  2016 Smaug.jpg
drwxrwxrwx 1 andre andre 4.0K Nov 29 00:16 __pycache__
-rwxrwxrwx 1 andre andre  43M Nov 29 00:33 dino-dragon-model.tflite
-rwxrwxrwx 1 andre andre  86M Nov 19 13:23 dino_dragon_10_0.899.h5
-rwxrwxrwx 1 andre andre  18K Nov 29 00:32 homework.ipynb
-rwxrwxrwx 1 andre andre 4.8K Nov 25 18:07 homework.md
-rwxrwxrwx 1 andre andre 1.2K Nov 29 00:25 lambda_function.py
-rwxrwxrwx 1 andre andre  220 Nov 29 00:30 push_to_ecr.sh
-rwxrwxrwx 1 andre andre  242 Nov 29 00:19 test.py


Answer: Model is 43 Mb

### Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24

In [6]:
import tensorflow.lite as tflite

In [7]:
interpreter = tflite.Interpreter(model_path='dino-dragon-model.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

Answer: 13

In [9]:
### Preparing the image
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [10]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
img = download_image(url)
img = prepare_image(img, (150, 150))

/tmp/ipykernel_16354/1511810874.py:18: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


### Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3353411
* 0.5529412
* 0.7458824
* 0.9654902

In [11]:
x = np.array(img, dtype='float32')

x /= 255

x[0][0]

array([0.5529412 , 0.31764707, 0.1764706 ], dtype=float32)

Answer: 0.5529412

### Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.17049132
* 0.39009996
* 0.60146114
* 0.82448614

In [12]:
X = [x]

In [13]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [14]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.82448506]], dtype=float32)

Answer: 0.82448506

### Prepare the lambda code

See [lambda_function.py](lambda_function.py)

### Docker

See [Dockerfile](Dockerfile)

## Question 5

Download the base image `svizor42/zoomcamp-dino-dragon-lambda:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 139 Mb
* 329 Mb
* 639 Mb
* 929 Mb

In [15]:
!docker pull svizor42/zoomcamp-dino-dragon-lambda:v2

v2: Pulling from svizor42/zoomcamp-dino-dragon-lambda
Digest: sha256:46be01f7953cd48ebe526329ddf781d92019b9d44de4ecb67d11f56eaccf79cb
Status: Image is up to date for svizor42/zoomcamp-dino-dragon-lambda:v2
docker.io/svizor42/zoomcamp-dino-dragon-lambda:v2


In [16]:
!docker images | grep svizor42

svizor42/zoomcamp-dino-dragon-lambda                                                 v2                               20ef58b21a05   10 days ago      639MB


Answer: 639 Mb

### Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `dino-vs-dragon-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

What's the output from the model?

* 0.12
* 0.32
* 0.52
* 0.72

In [17]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg'}

result = requests.post(url, json=data).json()
print(result)

[0.31950676441192627]


Answer: 0.32